In [187]:
import os
import bs4
import numpy as np
from typing import Literal
from langchain.text_splitter import RecursiveCharacterTextSplitter
from dotenv import load_dotenv
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.runnables import RunnablePassthrough,RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
import tiktoken
from langchain.load import loads, dumps
from operator import itemgetter
load_dotenv()
langchain_api = os.getenv('LANGCHAIN_API_KEY')
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'

In [116]:
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(parse_only =bs4.SoupStrainer( class_=("post-content", "post-title", "post-header" )))
)

In [117]:
docs= loader.load()

In [118]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

In [119]:
out = vectorstore._collection.get(include=["documents", "metadatas"])
print(out.keys())

for i in range(min(5,len(out['ids']))):
    print("ID:", out["ids"][i])
    print("Text:", out["documents"][i][:200])
    print("Meta:", out["metadatas"][i])
    print("="*50)

dict_keys(['ids', 'embeddings', 'documents', 'uris', 'included', 'data', 'metadatas'])
ID: 910181a3-dee0-44c8-af8f-d3b934ecc9bb
Text: LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


Building agents with LLM (large language model) as its core controller is a cool con
Meta: {'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}
ID: dc146721-f5f7-418e-82d7-83f8601ea8ff
Text: Memory

Short-term memory: I would consider all the in-context learning (See Prompt Engineering) as utilizing short-term memory of the model to learn.
Long-term memory: This provides the agent with th
Meta: {'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}
ID: b767d5e0-ca36-42b0-9fe5-535ba6928c0e
Text: Component One: Planning#
A complicated task usually involves many steps. An agent needs to know what they are and plan ahead.
Task Decomposition#
Chain of thought (CoT; Wei et al. 2022) has become a s
Meta: {'source': 'https

In [120]:
vectorstore._collection.count()

189

In [121]:
retriever = vectorstore.as_retriever()
prompt = ChatPromptTemplate.from_template(
    "Use the context to answer the question.\n\nContext:\n{context}\n\nQuestion: {question}"
)
print(prompt)

input_variables=['context', 'question'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Use the context to answer the question.\n\nContext:\n{context}\n\nQuestion: {question}'), additional_kwargs={})]


In [122]:
llm = ChatOpenAI(model_name = 'gpt-3.5-turbo',temperature=0)
def format_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What is Task Decomposition?")

'Task decomposition is a technique used to break down complex tasks into smaller and simpler steps, making them more manageable. This process can be done through prompting techniques like Chain of Thought (CoT) or Tree of Thoughts, task-specific instructions, or with the help of external classical planners like in the LLM+P approach.'

In [123]:
question = "Do you think Amir is going to become very rich by the age 40 years old?"
document = 'Amir will is going to gain networth of 200 million by the time he is 40 years ols. His income will get to at least 1 million per month.'

In [124]:
encoding = tiktoken.get_encoding("cl100k_base")
len(encoding.encode(question))

18

In [125]:
embd = OpenAIEmbeddings()
query_result = embd.embed_query(question) 
document_result = embd.embed_documents([document])[0]

In [126]:
def cosine(vec1,vec2):
    dot_product = np.dot(vec1,vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    return dot_product / (norm_vec1 * norm_vec2)

In [127]:
cosine(query_result,document_result)

np.float64(0.9074268061127665)

In [128]:
prompt = """ Answer the question based on the following contex:
{context}

Question: {question}

"""
prompt = ChatPromptTemplate.from_template(prompt)
llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0)


In [129]:
chain = prompt | llm | StrOutputParser()
chain.invoke({'context':docs, 'question':'What is Task Decomposition'})

'Task decomposition in a LLM-powered autonomous agent system involves breaking down large tasks into smaller, more manageable subgoals. This process enables the agent to efficiently handle complex tasks by dividing them into simpler steps. Task decomposition can be achieved through various methods, such as using simple prompting for LLM, task-specific instructions, or human inputs. By breaking down tasks into smaller components, the agent can effectively plan and execute its actions to achieve the overall goal.'

In [130]:
rag_chain = (
    {'context': retriever, 'question': RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke('What is Task Decomposition')

"Task decomposition is a technique used by agents to break down complex tasks into smaller and simpler steps. This process involves transforming big tasks into multiple manageable tasks, allowing the agent to plan ahead and interpret the model's thinking process. Techniques such as Chain of Thought (CoT) and Tree of Thoughts extend task decomposition by exploring multiple reasoning possibilities at each step and generating multiple thoughts per step, creating a tree structure."

#### Multi Query

In [131]:
template = """You are an AI language model assistant. Your task is to generate five 
different versions of the given user question to retrieve relevant documents from a vector 
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search. 
Provide these alternative questions separated by newlines. Original question: {question}"""
prompt = ChatPromptTemplate.from_template(template)
generate_queries = (
      prompt
    | ChatOpenAI(model='gpt-3.5-turbo',temperature=0)
    | StrOutputParser()
    | (lambda x:x.split('\n'))
)

def get_unique_union(documents):
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    unique_docs = list(set(flattened_docs))
    # Return
    return [loads(doc) for doc in unique_docs]
    

# Testing

# queries = generate_queries.invoke("what is the Decomposition in LLM")
# batches = [retriever.get_relevant_documents(q) for q in queries]
# unique_docs = get_unique_union(batches)

question = "What is task decomposition for LLM agents?"
retriever_chain = generate_queries| retriever.map() | get_unique_union
docs = retriever_chain.invoke({"question":question})

In [132]:
template = """Answer the following question based on this context:

{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

llm = ChatOpenAI(temperature=0)

final_rag_chain = ( {'context': retriever_chain,
                     'question': itemgetter('question')}
                   |prompt
                   |llm|StrOutputParser()
)

final_rag_chain.invoke({'question':question})

'Task decomposition for LLM agents involves breaking down complex tasks into smaller and simpler steps using techniques like Chain of Thought (CoT) and Tree of Thoughts. Additionally, LLM agents can also rely on external classical planners for long-horizon planning using the Planning Domain Definition Language (PDDL) as an intermediate interface.'

#### RAG Fusion

In [133]:
template = """You are a helpful assistant that generates multiple search queries based on a single input query. \n
Generate multiple search queries related to: {question} \n
Output (4 queries):"""
prompt_rag_fusion = ChatPromptTemplate.from_template(template)

generate_queries = (
    prompt_rag_fusion
    |ChatOpenAI(model = 'gpt-3.5-turbo',temperature=0)
    |StrOutputParser()
    |(lambda x: x.split('\n'))
    
)

# Testing
# results = generate_queries.invoke({'question':"what is decomposition in LLM"})

def reciprocal_rank_fusion(results:list[list],k=60):

    fused_score ={}

    for docs in results:
        for rank,doc in enumerate(docs):
            doc_str = dumps(doc)
            fused_score[doc_str] = fused_score.get(doc_str,0)+1 /(rank+k)

    reranked_items =[(loads(doc),score) for doc ,score in sorted(fused_score.items(), key=lambda x:x[1],reverse=True)]
    return reranked_items

retrieval_chain = (generate_queries
                   |retriever.map()
                   |reciprocal_rank_fusion
)
question = "What is task decomposition for LLM agents?"
docs = retrieval_chain.invoke({'question':question})


template = """
Answer the following question based on this context:

{context}

Question:{question}

"""

prompt = ChatPromptTemplate.from_template(template)

fusion_chain = ( {'context':retrieval_chain, 'question':RunnablePassthrough()}
                |prompt
                |llm
                |StrOutputParser()

)
fusion_chain.invoke(question)

'Task decomposition for LLM agents involves breaking down complex tasks into smaller and simpler steps using techniques such as prompting, task-specific instructions, and human inputs. Additionally, a distinct approach known as LLM+P involves outsourcing the planning step to an external classical planner that utilizes the Planning Domain Definition Language (PDDL) to describe the planning problem and generate a plan that is then translated back into natural language. This allows LLM agents to effectively plan and execute tasks by leveraging external tools and domain-specific knowledge.'

#### Decomposition

In [134]:
# Decomposition
template = """You are a helpful assistant that generates multiple sub-questions related to an input question. \n
The goal is to break down the input into a set of sub-problems / sub-questions that can be answers in isolation. \n
Generate multiple search queries related to: {question} \n
Output (3 queries):"""
prompt_decomposition = ChatPromptTemplate.from_template(template)

generate_queries_decomposition =(
    prompt_decomposition
    |llm
    |StrOutputParser()
    |(lambda x:x.split('\n'))
)
question = "What are the main components of an LLM-powered autonomous agent system?"
questions = generate_queries_decomposition.invoke(question)


Answer recursively

In [135]:
template = """
here is the question you need to answer:
\n --- \n {question} \n --- \n

Here is any available background question + answer pairs:

\n --- \n {q_a_pairs} \n --- \n

Here is additional context relevant to the question: 

\n --- \n {context} \n --- \n

Use the above context and any background question + answer pairs to answer the question: \n {question}
"""


decomposition_prompt = ChatPromptTemplate.from_template(template)



In [136]:
def format_qa_pair(question, answer):
    """Format Q and A pair"""
    
    formatted_string = ""
    formatted_string += f"Question: {question}\nAnswer: {answer}\n\n"
    return formatted_string.strip()


q_a_pairs = ""

for q in questions:
    rag_chain =({
        'context': itemgetter('question') |retriever,
        'question': itemgetter('question'),
        'q_a_pairs': itemgetter('q_a_pairs')}
        |decomposition_prompt
        |llm
        |StrOutputParser()
    )
    answer = rag_chain.invoke({'question':q,'q_a_pairs':q_a_pairs})
    answer = rag_chain.invoke({"question":q,"q_a_pairs":q_a_pairs})
    q_a_pair = format_qa_pair(q,answer)
    q_a_pairs = q_a_pairs + "\n---\n"+  q_a_pair

Answer individually

In [137]:
template = """You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Use three sentences maximum and keep the answer concise.

\nQuestion: {question} \nContext: {context} \nAnswer:"""

prompt_rag = ChatPromptTemplate.from_template(template)

def retrieve_rag_chain(question,prompt,chain):
    sub_questions = chain.invoke(question)

    rag_results =[]

    for q in sub_questions:
        retrieved = retriever.get_relevant_documents(q)
        answer =(prompt_rag | llm | StrOutputParser()).invoke({"context": retrieved, 
                                                                "question": q})
        rag_results.append(answer)
        
    return questions,sub_questions
questions,answers = retrieve_rag_chain(question,prompt_rag,generate_queries_decomposition)


def format_qa_paris(questions, answers):

    fromat_strings=''

    for i, (questions,answers) in enumerate(zip(questions,answers),start=1):
         fromat_strings += f"Question {i}: {question}\nAnswer {i}: {answer}\n\n"
    return fromat_strings.strip()  

question = "What are the main components of an LLM-powered autonomous agent system?"

context = format_qa_paris(questions,answers)

template = """Here is a set of Q+A pairs:

{context}

Use these to synthesize an answer to the question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)


final_rag_chain=(
    prompt
    |llm
    |StrOutputParser()
).invoke({'context':context, 'question':question})
final_rag_chain

'The main components of an LLM-powered autonomous agent system include the LLM as the core controller, planning for task breakdown, subgoal decomposition for task management, reflection for self-criticism and learning, refinement for problem-solving enhancement, and memory for information storage and retrieval. These components work together to enable the autonomous agent system to function effectively, solve complex problems, and achieve autonomy in decision-making and task execution.'

#### Step Back

In [175]:
examples = [
    {
        "input": "Could the members of The Police perform lawful arrests?",
        "output": "what can the members of The Police do?",
    },
    {
        "input": "Jan Sindel’s was born in what country?",
        "output": "what is Jan Sindel’s personal history?",
    },
]

example_prompt = ChatPromptTemplate.from_messages([
    ("human","{input}"),
    ("ai","{output}")
])

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

prompt = ChatPromptTemplate.from_messages([
    (
     "system", """You are an expert at world knowledge. Your task is to step back and paraphrase a question to a more generic step-back question, which is easier to answer. Here are a few examples:""",   
    ),
    few_shot_prompt,
    ("user","{question}")
    ]
)

# # Testing
generate_queries_step_back = (prompt|llm|StrOutputParser())
# generate_queries_step_back

response_prompt_template = """You are an expert of world knowledge. I am going to ask you a question. Your response should be comprehensive and not contradicted with the following context if they are relevant. Otherwise, ignore them if they are not relevant.

# {normal_context}
# {step_back_context}

# Original Question: {question}
# Answer:"""

response_prompt = ChatPromptTemplate.from_template(response_prompt_template)

chain = (
    {"normal_context": itemgetter('question')|retriever,
          "step_back_context":generate_queries_step_back |retriever,
          'question': itemgetter('question')}

          |response_prompt
          |llm|StrOutputParser()


 )

In [180]:
print(chain.invoke({'question': "what is decomposition in LLM"}))

Decomposition in LLM refers to the process of breaking down complex tasks into smaller, more manageable subgoals. This allows the LLM (large language model) to efficiently handle intricate tasks by dividing them into smaller steps. There are several ways in which task decomposition can be achieved within the context of LLM:

1. **LLM with Simple Prompting**: One approach to task decomposition involves using simple prompts to guide the LLM in breaking down tasks. For example, prompts like "Steps for XYZ" or "What are the subgoals for achieving XYZ?" can help the LLM organize the task into smaller components.

2. **Task-Specific Instructions**: Another method is to provide task-specific instructions to the LLM. For instance, instructing the LLM to "Write a story outline" when the task involves creating a novel can help in task decomposition tailored to the specific requirements of the task.

3. **Human Inputs**: Task decomposition can also be achieved through human inputs, where individu

#### HyDE

In [184]:
template = """Please write a scientific paper passage to answer the question
Question: {question}
Passage:"""
prompt_hyde = ChatPromptTemplate.from_template(template)

from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

generate_docs_for_retrieval = (
    prompt_hyde | ChatOpenAI(temperature=0) | StrOutputParser() 
)

# Run
question = "What is task decomposition for LLM agents?"
generate_docs_for_retrieval.invoke({"question":question})

retrieval_chain = generate_docs_for_retrieval | retriever 
retrieved_docs = retrieval_chain.invoke({"question":question})

template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    prompt
    | llm
    | StrOutputParser()
)

print(final_rag_chain.invoke({"context":retrieved_docs,"question":question}))

Task decomposition for LLM agents can be done by using simple prompting like "Steps for XYZ", "What are the subgoals for achieving XYZ?", task-specific instructions, or with human inputs. Additionally, there is a distinct approach called LLM+P that involves relying on an external classical planner to do long-horizon planning using the Planning Domain Definition Language (PDDL) as an intermediate interface.


#### Routing

In [199]:
class RouteQuery(BaseModel):
    datasource : Literal["python_docs", "js_docs", "golang_docs"] = Field(
        ...,
        description="Given a user Question choose which datasource would be the most relevant for answering their question"
    )


structured_llm = llm.with_structured_output(RouteQuery)
system_prompt =  """You are an expert at routing a user question to the appropriate data source.

Based on the programming language the question is referring to, route it to the relevant data source."""

question = """Why doesn't the following code work:

from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(["human", "speak in {language}"])
prompt.invoke("french")
"""
prompt = ChatPromptTemplate.from_messages([
    ('system',system_prompt),
    ('user',"{question}")
])

router = (prompt | structured_llm)


result = router.invoke({'question':question})
result

/opt/anaconda3/envs/pytorch/lib/python3.12/site-packages/langchain_openai/chat_models/base.py:1900: UserWarning: Received a Pydantic BaseModel V1 schema. This is not supported by method="json_schema". Please use method="function_calling" or specify schema via JSON Schema or Pydantic V2 BaseModel. Overriding to method="function_calling".
  warnings.warn(
/opt/anaconda3/envs/pytorch/lib/python3.12/site-packages/langchain_openai/chat_models/base.py:1913: UserWarning: Cannot use method='json_schema' with model gpt-3.5-turbo-0125 since it doesn't support OpenAI's Structured Output API. You can see supported models here: https://platform.openai.com/docs/guides/structured-outputs#supported-models. To fix this warning, set `method='function_calling'. Overriding to method='function_calling'.
  warnings.warn(


RouteQuery(datasource='python_docs')